<span style="color:hotpink; font-size:40px; font-weight:bold;">Packages and imports</span>

In [39]:
### standard imports ###

import numpy as np
import pandas as pd
import xarray as xr
import gcsfs

### Python file with supporting functions ###

import residual_utils as prerun

### set up for getting files from leap bucket ###

fs = gcsfs.GCSFileSystem()

<span style="color:hotpink; font-size:40px; font-weight:bold;">Setting date range</span>

In [3]:
### Setting the date range to unify the date type ###

# Define date range
date_range_start = '1982-02-01T00:00:00.000000000'
date_range_end = '2023-12-31T00:00:00.000000000'

# create date vector, adds 14 days to start & end
dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS')

<span style="color:hotpink; font-size:40px; font-weight:bold;">Setting paths</span>

In [40]:
### set paths ###

### paths for loading: ###
# directory of regridded members from notebook 00
ensemble_dir = 'gs://leap-persistent/mauriekeppens/Ensemble_Testbed_Analysis/00_regridded_members' # includes 121 members
# directory of reference zarr files
zarr_dir = 'gs://leap-persistent/abbysh/zarr_files_'

### loading: ###
# atmospheric xco2 file
xco2_path = f"{zarr_dir}/xco2_cmip6_183501-224912_monthstart.zarr"
# socat data file
socat_path = f"{zarr_dir}/socat_mask_feb1982-dec2023.zarr" 
            # this is the SOCAT mask that is used to make the "testbed-truth' -> consistent with the 'real-world' SOCAT coverage (so that the ML reconstructions are based on real-world pCO2 coverages)
            # testbed-truth -> we have every 1°x1° monthly a pCO2 value (because of the ESM member pCO2 estimates) -> using that as the "truth" so we have the entire world to validate the ML against
            # if we would use only the real pCO2 SOCAT data, we wouldn't even be able to validate the performance of the ML products in the southern ocean (becasuse super little data there)
# topo and land-sea masks
topo_path = f"{zarr_dir}/GEBCO_2014_1x1_global.zarr"
lsmask_path = f"{zarr_dir}/lsmask.zarr"

#############################################

### paths for saving: ###
your_username = "mauriekeppens
# directory of where to save ML inputs from this (01) notebook (save it in your bucket!!)
xgb_data_dir = f"gs://leap-persistent/{your_username}/Ensemble_Testbed_Analysis/01_ML_inputs" 

<span style="color:hotpink; font-size:40px; font-weight:bold;">Calculating chlorophyll climatology</span>

Here is where clorophyll climatology is calculated. Climatology is calculated from model output 01-1998 through the end of the time period (here, 12-2023), by calculating the monthly average chlorophyll of the whole time period. This climatology is saved in a separate file in the individual regridded member folders. This file is currently used to set feature variables for ML in notebook 02, but those feature variables are not *actually used* for the ML. 

You do not need to run this code, because chlorophyll climatology is already saved in the regridded members directory that you're loading from.

In [5]:
### chl_clim code here ###

a = fs.ls(ensemble_dir)
for ens_path in a[1:]:
    ens = ens_path.split('/')[-1] # goes through all the ESM
    mems = fs.ls(ens_path) # all members in that specific ESM
    for mem in mems:
        memo = mem.split('/')[-1]
        zarrs = fs.ls(mem)
        for z in zarrs: # Loop through Zarr datasets for each member
            zarr_name = z.split('/')[-1]
            d = xr.open_mfdataset('gs://'+z,engine='zarr')
            chl = d.chl # Open Zarr dataset and extract chlorophyll

            ### year selection ### remove hard code ###
            chl_clim = chl.sel(time=slice('1998-01-01', '2023-12-30')).groupby("time.month").mean("time") # group by month and take the mean over the time (dimension : 12 x lat x lon, each month)

            if 'lev_partial' in chl_clim.dims: # If the dataset has a vertical dimension lev_partial, only keeps the first level.
                chl_clim = chl_clim.sel({'lev_partial':1},drop=True)

            # Makes a new dataset with dimensions month x lat x lon for the monthly climatology (12 x lat x lon)
            chl_out_new = xr.Dataset({'chl_clim':(["month","lat","lon"],chl_clim.data)}, 
                    coords={'time': (['month'],chl_clim.month.data),
                            'lon':(['lon'],chl_clim.xlon.data),
                            'lat': (['lat'],chl_clim.ylat.data)})
            
            # print(chl_out_new)
            out_path = f'{ensemble_dir}/{ens}/{memo}/chlclim_{zarr_name}' # "chlclim" added to the name so we know this is the climatology chl
            print(out_path,chl_out_new)
            chl_out_new.to_zarr(out_path) # add this zarr file to the ensemble_member zarr file containing all variables (SST, SSS, chla, MLD, pCO2, pCO2_T, pCO2_residual)
            print(f'finished with {ens}:{memo}')

<span style="color:hotpink; font-size:40px; font-weight:bold;">Loading list of ESMs and members in testbed</span>

<span style="color:lightblue; font-size:30px; font-weight:bold;">For all members for each ESM</span>

In [6]:
### loads list of Earth System Models ("ensembles") and members for the full testbed ###

ensembles = []
mems_dict = dict()

# list of all ensembles in that ensemble_dir
for path in fs.ls(ensemble_dir):
    ens_name = path.split('/')[-1] 
    if ens not in ensembles: # Checks if the ensemble is already in the list; if not, adds it.
        ensembles.append(ens_name)

        # members of that ensemble
        members = []
        for mem_path in fs.ls(path): # find members of that ensemble
            mem_name = mem_path.split('/')[-1]
            members.append(mem_name)
        mems_dict[ens_name] = members # Adds the member names to the dictionary under that ensemble

# Print results
print("Ensembles found:")
print(ensembles)
print("\nMembers for each ensemble:")
for ens, members in mems_dict.items():
    print(f"{ens}: {members}")

Ensembles found:
['ACCESS-ESM1-5', 'CESM2', 'CESM2-WACCM', 'CMCC-ESM2', 'CanESM5', 'CanESM5-CanOE', 'GFDL-ESM4', 'MPI-ESM1-2-LR', 'UKESM1-0-LL']

Members for each ensemble:
ACCESS-ESM1-5: ['member_r10i1p1f1', 'member_r15i1p1f1', 'member_r17i1p1f1', 'member_r1i1p1f1', 'member_r22i1p1f1', 'member_r26i1p1f1', 'member_r27i1p1f1', 'member_r2i1p1f1', 'member_r31i1p1f1', 'member_r32i1p1f1', 'member_r33i1p1f1', 'member_r34i1p1f1', 'member_r35i1p1f1', 'member_r36i1p1f1', 'member_r37i1p1f1', 'member_r38i1p1f1', 'member_r39i1p1f1', 'member_r3i1p1f1', 'member_r40i1p1f1', 'member_r4i1p1f1', 'member_r5i1p1f1', 'member_r7i1p1f1', 'member_r8i1p1f1']
CESM2: ['member_r10i1p1f1', 'member_r11i1p1f1', 'member_r4i1p1f1']
CESM2-WACCM: ['member_r1i1p1f1', 'member_r2i1p1f1', 'member_r3i1p1f1']
CMCC-ESM2: ['member_r1i1p1f1']
CanESM5: ['member_r10i1p1f1', 'member_r10i1p2f1', 'member_r1i1p1f1', 'member_r1i1p2f1', 'member_r2i1p1f1', 'member_r2i1p2f1', 'member_r3i1p1f1', 'member_r3i1p2f1', 'member_r4i1p1f1', 'membe

In [ ]:
# total number of members
total_members = sum(len(members) for members in mems_dict.values())
print(f"Total number of members: {total_members}")

# number of members per ensemble
for ens, members in mems_dict.items():
    print(f"{ens}: {len(members)} members")

<span style="color:hotpink; font-size:40px; font-weight:bold;">Running code to make ML inputs</span>

In [18]:
### creating pandas dataframes out of "raw" data to prep for ML ###
import os

N_time = len(dates)
member_counter = 0 

### loop through each ESM
for ens, mem_list in mems_dict.items(): 

    ### loop through each member of that ESM
    for member in mem_list:
        
        init_date = str(dates[0].year) + format(dates[0].month,'02d')
        fin_date  = str(dates[-1].year) + format(dates[-1].month,'02d')
        
        output_dir = f"{xgb_data_dir}/{ens}/{member}"
        save_file = f"{output_dir}/MLinput_{ens}_{member.split('_')[-1]}_mon_1x1_{init_date}_{fin_date}.pkl"
        
        if fs.exists(save_file):
            print(f"Skipping {ens}, {member} (already exists)")
            continue

        print(f"Making dataframe {member_counter}: {ens}, {member}")
        
        ### uses utility function file to make data into dataframe for ML use
        df = prerun.create_inputs(ensemble_dir, ens, member, dates, N_time,
                                  xco2_path,
                                  socat_path,
                                  topo_path,
                                  lsmask_path)

        ### Save the pandas dataframe to workspace
        prerun.save_clean_data(df, xgb_data_dir, ens, member, dates)
        member_counter += 1

Skipping ACCESS-ESM1-5, member_r10i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r15i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r17i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r1i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r22i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r26i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r27i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r2i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r31i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r32i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r33i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r34i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r35i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r36i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r37i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r38i1p1f1 (already exists)
Skipping ACCESS-ESM1-5, member_r39i1p1f1 (already exists)
Skipping ACCESS-

In [28]:
fs.ls(f'gs://leap-persistent/{your_username}/pco2_residual/post01_xgb_inputs')

FileNotFoundError: b/leap-persistent/o/abbysh%2Fpco2_residual%2Fpost01_xgb_inputs

In [41]:
fs.ls(f'gs://leap-persistent/{your_username}/pco2_residual/post01_xgb_inputs')

['leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/ACCESS-ESM1-5',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CESM2',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CESM2-WACCM',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CMCC-ESM2',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CanESM5',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CanESM5-CanOE',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/GFDL-ESM4',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/MPI-ESM1-2-LR',
 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/UKESM1-0-LL']

In [42]:
# Check whether we have all members saved
import gcsfs

fs = gcsfs.GCSFileSystem()

base_path = f'gs://leap-persistent/{your_username}/pco2_residual/post01_xgb_inputs'
ensembles = fs.ls(base_path)
print("Ensembles found:", ensembles)

total_members = 0
for ens_path in ensembles:
    members = fs.ls(ens_path)
    print(f"{ens_path} members:", [m.split('/')[-1] for m in members])
    total_members += len(members)

print("Total members:", total_members)

Ensembles found: ['leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/ACCESS-ESM1-5', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CESM2', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CESM2-WACCM', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CMCC-ESM2', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CanESM5', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/CanESM5-CanOE', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/GFDL-ESM4', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/MPI-ESM1-2-LR', 'leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/UKESM1-0-LL']
leap-persistent/mauriekeppens/pco2_residual/post01_xgb_inputs/ACCESS-ESM1-5 members: ['member_r10i1p1f1', 'member_r15i1p1f1', 'member_r17i1p1f1', 'member_r1i1p1f1', 'member_r22i1p1f1', 'member_r26i1p1f1', 'member_r27i1p1f1', 'member_r2i1p1f1', 'member_r31i1p1f1', 'member_r32i1p1f1', 'member_r33i1